# Serialization semantics
This note describes how you can save and load PyTorch tensors and module states in Python, and how to serialize Python modules so they can be loaded in C++.

In [1]:
import torch

## Saving and loading tensors

torch.save() and torch.load() let you easily save and load tensors:

In [2]:
t = torch.tensor([1., 2.])
torch.save(t, 'tensor.pt')

In [3]:
loaded_t = torch.load('tensor.pt')
print(loaded_t)

tensor([1., 2.])


By convention, PyTorch files are typically written with a ‘.pt’ or ‘.pth’ extension.

torch.save() and torch.load() use Python’s pickle by default, so you can also save multiple tensors as part of Python objects like tuples, lists, and dicts:

In [4]:
d = {'a': torch.tensor([1., 2.]), 'b': torch.tensor([3., 4.])}
torch.save(d, 'tensor_dict.pt')

In [5]:
loaded_d = torch.load('tensor_dict.pt')
print(loaded_d)

{'a': tensor([1., 2.]), 'b': tensor([3., 4.])}


> Custom data structures that include PyTorch tensors can also be saved if the data structure is pickle-able

## Saving and loading tensors preserves views

In [6]:
numbers = torch.arange(1, 10)
print(numbers)
evens = numbers[1::2]
print(evens)
torch.save([numbers, evens], 'tensors.pt')

tensor([1, 2, 3, 4, 5, 6, 7, 8, 9])
tensor([2, 4, 6, 8])


In [7]:
loaded_numbers, loaded_evens = torch.load('tensors.pt')
print(loaded_numbers)
print(loaded_evens)

tensor([1, 2, 3, 4, 5, 6, 7, 8, 9])
tensor([2, 4, 6, 8])


In [8]:
loaded_evens *= 2
print(loaded_numbers)

tensor([ 1,  4,  3,  8,  5, 12,  7, 16,  9])


Behind the scenes, these tensors share the same “storage.” See Tensor Views for more on views and storage.

When PyTorch saves tensors it saves their storage objects and tensor metadata separately. This is an implementation detail that may change in the future, but it typically saves space and lets PyTorch easily reconstruct the view relationships between the loaded tensors. In the above snippet, for example, only a single storage is written to ‘tensors.pt’.

In some cases, however, saving the current storage objects may be unnecessary and create prohibitively large files. In the following snippet a storage much larger than the saved tensor is written to a file:

In [9]:
large = torch.arange(1, 1000)
small = large[0:5]
torch.save(small, 'small.pt')
loaded_small = torch.load('small.pt')
loaded_small.storage().size()

999

Instead of saving only the five values in the small tensor to ‘small.pt,’ the 999 values in the storage it shares with large were saved and loaded.

When saving tensors with fewer elements than their storage objects, the size of the saved file can be reduced by first cloning the tensors. Cloning a tensor produces a new tensor with a new storage object containing only the values in the tensor:

In [10]:
large = torch.arange(1, 1000)
small = large[0:5]
torch.save(small.clone(), 'small.pt')  # saves a clone of small
loaded_small = torch.load('small.pt')
loaded_small.storage().size()

5

Since the cloned tensors are independent of each other, however, they have none of the view relationships the original tensors did. If both file size and view relationships are important when saving tensors smaller than their storage objects, then care must be taken to construct new tensors that minimize the size of their storage objects but still have the desired view relationships before saving.

# Saving and loading torch.nn.Modules

In PyTorch, a module’s state is frequently serialized using a ‘state dict.’ A module’s state dict contains all of its parameters and persistent buffers:

In [11]:
bn = torch.nn.BatchNorm1d(3, track_running_stats=True)
print(list(bn.named_parameters()))

print(list(bn.named_buffers()))

print(bn.state_dict())

[('weight', Parameter containing:
tensor([1., 1., 1.], requires_grad=True)), ('bias', Parameter containing:
tensor([0., 0., 0.], requires_grad=True))]
[('running_mean', tensor([0., 0., 0.])), ('running_var', tensor([1., 1., 1.])), ('num_batches_tracked', tensor(0))]
OrderedDict([('weight', tensor([1., 1., 1.])), ('bias', tensor([0., 0., 0.])), ('running_mean', tensor([0., 0., 0.])), ('running_var', tensor([1., 1., 1.])), ('num_batches_tracked', tensor(0))])


Instead of saving a module directly, for compatibility reasons it is recommended to instead save only its state dict. Python modules even have a function, load_state_dict(), to restore their states from a state dict:

In [12]:
torch.save(bn.state_dict(), 'bn.pt')
bn_state_dict = torch.load('bn.pt')
new_bn = torch.nn.BatchNorm1d(3, track_running_stats=True)
new_bn.load_state_dict(bn_state_dict)

<All keys matched successfully>

Note that the state dict is first loaded from its file with torch.load() and the state then restored with load_state_dict().

Even custom modules and modules containing other modules have state dicts and can use this pattern:

In [13]:
class MyModule(torch.nn.Module):
    def __init__(self):
        super(MyModule, self).__init__()
        self.l0 = torch.nn.Linear(4, 2)
        self.l1 = torch.nn.Linear(2, 1)

    def forward(self, input):
        out0 = self.l0(input)
        out0_relu = torch.nn.functional.relu(out0)
        return self.l1(out0_relu)

m = MyModule()
print(m.state_dict())

torch.save(m.state_dict(), 'mymodule.pt')

OrderedDict([('l0.weight', tensor([[-0.1100, -0.2925,  0.4771,  0.4324],
        [-0.4550,  0.0150, -0.3119,  0.3826]])), ('l0.bias', tensor([-0.0250,  0.1102])), ('l1.weight', tensor([[-0.6759,  0.3887]])), ('l1.bias', tensor([-0.1171]))])


In [14]:
m_state_dict = torch.load('mymodule.pt')
new_m = MyModule()
new_m.load_state_dict(m_state_dict)

<All keys matched successfully>

# Serializing torch.nn.Modules and loading them in C++

ScriptModules can be serialized as a TorchScript program and loaded using torch.jit.load(). This serialization encodes all the modules’ methods, submodules, parameters, and attributes, and it allows the serialized program to be loaded in C++ (i.e. without Python).


The distinction between torch.jit.save() and torch.save() may not be immediately clear. torch.save() saves Python objects with pickle. This is especially useful for prototyping, researching, and training. torch.jit.save(), on the other hand, serializes ScriptModules to a format that can be loaded in Python or C++. This is useful when saving and loading C++ modules or for running modules trained in Python with C++, a common practice when deploying PyTorch models.

To script, serialize and load a module in Python:

In [15]:
scripted_module = torch.jit.script(MyModule())
torch.jit.save(scripted_module, 'mymodule.pt')
torch.jit.load('mymodule.pt')

RecursiveScriptModule(
  original_name=MyModule
  (l0): RecursiveScriptModule(original_name=Linear)
  (l1): RecursiveScriptModule(original_name=Linear)
)

Traced modules can also be saved with torch.jit.save(), with the caveat that only the traced code path is serialized. The following example demonstrates this:

In [16]:
class ControlFlowModule(torch.nn.Module):
    def __init__(self):
        super(ControlFlowModule, self).__init__()
        self.l0 = torch.nn.Linear(4, 2)
        self.l1 = torch.nn.Linear(2, 1)

    def forward(self, input):
        if input.dim() > 1:
            return torch.tensor(0)

        out0 = self.l0(input)
        out0_relu = torch.nn.functional.relu(out0)
        return self.l1(out0_relu)


traced_module = torch.jit.trace(ControlFlowModule(), torch.randn(4))
torch.jit.save(traced_module, 'controlflowmodule_traced.pt')
loaded = torch.jit.load('controlflowmodule_traced.pt')
print(loaded(torch.randn(2, 4)))

scripted_module = torch.jit.script(ControlFlowModule(), torch.randn(4))
torch.jit.save(scripted_module, 'controlflowmodule_scripted.pt')
loaded = torch.jit.load('controlflowmodule_scripted.pt')
print(loaded(torch.randn(2, 4)))

tensor([[-0.3560],
        [-0.2269]], grad_fn=<AddmmBackward0>)
tensor(0)


/home/sz/anaconda3/envs/cling/lib/python3.10/site-packages/torch/jit/_script.py:1243: UserWarning: `optimize` is deprecated and has no effect. Use `with torch.jit.optimized_execution() instead
  warnings.warn(


The above module has an if statement that is not triggered by the traced inputs, and so is not part of the traced module and not serialized with it. The scripted module, however, contains the if statement and is serialized with it. See the TorchScript documentation for more on scripting and tracing.

Finally, to load the module in C++:
    
~~~
torch::jit::script::Module module;
module = torch::jit::load('controlflowmodule_scripted.pt');
~~~

## save tensors for loading from c++

In [18]:
class TensorContainer(torch.nn.Module):
    def __init__(self, tensor_dict):
        super().__init__()
        for key,value in tensor_dict.items():
            setattr(self, key, value)
 
x = torch.ones(4, 4)
tensor_dict = {'x': x}
tensors = TensorContainer(tensor_dict)
tensors = torch.jit.script(tensors)
tensors.save('x.pth')

~~~
// c++ code
try
{
    torch::jit::script::Module tensors = torch::jit::load("x.pth");
    c10::IValue iv = tensors.attr("x");
    torch::Tensor ts = iv.toTensor();
    std::cout << ts;
}catch (const c10::Error& e) {
    std::cerr << "error loading the tensor " << std::endl;
    std::cerr << e.msg() << std::endl;
}
~~~

# reference

https://pytorch.org/docs/stable/notes/serialization.html#

https://discuss.pytorch.org/t/load-tensors-saved-in-python-from-c-and-vice-versa/39435/8